In [ ]:
# import necessary modules
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
tf.random.set_seed(42)
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from random import randint

In [ ]:
# importing and labeling the data

data_path = 'Dog Emotion'

class_names=['angry', 'happy', 'relaxed', 'sad']
num_classes = len(class_names)

img_size = (192, 192, 3)


images = []
labels = []
labels_df = pd.read_csv('Dog Emotion\\labels.csv')


for image in labels_df.iloc:
    images.append(np.asarray(cv2.resize(cv2.imread(data_path+ '\\' + image[2] + '\\' + image[1], cv2.IMREAD_COLOR), img_size[0:2])[:, :, ::-1]))
    
    # converting labels into one-hot-encoding i.e., for sad label will be [0, 0, 0, 1]
    label = np.zeros(num_classes)
    label[class_names.index(image[2])] = 1
    labels.append(label)

labels = np.asarray(labels)
images = np.asarray(images)


In [ ]:
# display 16 pictures from the dataset

fig, axs = plt.subplots(4, 4, figsize=(11, 11))

for x in range(4):
    for y in range(4):
        i = randint(0, len(images))
        
        axs[x][y].imshow(images[i])
        
        # delete x and y ticks and set x label as picture label
        axs[x][y].set_xticks([])
        axs[x][y].set_yticks([])
        axs[x][y].set_xlabel(class_names[np.argmax(labels[i])])
        
plt.show()

In [ ]:
# diving the data into train-val-test sets

X_train, X_trail, y_train, y_trail = train_test_split(images, labels, test_size=0.20, random_state=42, stratify=labels, shuffle=True)
X_val, X_test, y_val, y_test = train_test_split(X_trail, y_trail, test_size=0.7, random_state=42, stratify=y_trail, shuffle=True)

print(f'train images shape: {X_train.shape}\ntrain labels shape: {y_train.shape}\n\nvalidation images shape: {X_val.shape}\nvalidation labels shape: {y_val.shape}\n\ntest images shape: {X_test.shape}\ntest labels shape: {y_test.shape}\n')

In [ ]:
# defining model architecture

cnn_model = tf.keras.Sequential()

# Inputs and rescaling
cnn_model.add(tf.keras.layers.Rescaling(scale=1. / 255, input_shape=(img_size)))

# Convolutional block 1
cnn_model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
cnn_model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
cnn_model.add(tf.keras.layers.MaxPooling2D(pool_size=2))

# Convolutional block 2
cnn_model.add(tf.keras.layers.Conv2D(128, (2, 2), activation='relu', padding='same'))
cnn_model.add(tf.keras.layers.Conv2D(128, (2, 2), activation='relu', padding='same'))
cnn_model.add(tf.keras.layers.MaxPooling2D(pool_size=2))

# Convolutional block 3
cnn_model.add(tf.keras.layers.Conv2D(256, (2, 2), activation='relu', padding='same'))
cnn_model.add(tf.keras.layers.Conv2D(256, (2, 2), activation='relu', padding='same'))
cnn_model.add(tf.keras.layers.MaxPooling2D(pool_size=2))

# Convolutional block 4
cnn_model.add(tf.keras.layers.Conv2D(512, (2, 2), activation='relu', padding='same'))
cnn_model.add(tf.keras.layers.Conv2D(512, (2, 2), activation='relu', padding='same'))
cnn_model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
cnn_model.add(tf.keras.layers.Flatten())

# Dense block
cnn_model.add(tf.keras.layers.Dense(256, activation='relu'))
cnn_model.add(tf.keras.layers.Dense(128, activation='relu'))
cnn_model.add(tf.keras.layers.Dense(64, activation='relu'))
cnn_model.add(tf.keras.layers.Dense(num_classes, activation='softmax'))


cnn_model.compile(optimizer='Adamax', loss='categorical_crossentropy', metrics=['accuracy'])

cnn_model.summary()

In [ ]:
# defining the callbacks
callback = EarlyStopping(monitor = "loss", patience = 5, 
            restore_best_weights = True, mode='min')

In [ ]:
# training the model for 25 epochs

history = cnn_model.fit(images, labels, epochs=25, validation_data=(X_val, y_val), callbacks=[callback])

In [ ]:
# plot the accuracy and loss curves

accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(accuracy))

plt.figure()
plt.plot(epochs, accuracy, label='Training Accuracy')
plt.plot(epochs, loss, label='Training Loss')
plt.legend()
plt.title('Training Accuracy and Loss')

plt.figure()
plt.plot(epochs, val_accuracy, label='Validation Accuracy')
plt.plot(epochs, val_loss, label='Validation Loss')
plt.legend()
plt.title('Validation Accuracy and Loss')

plt.show()

In [ ]:
# evaluating the accuracy and loss in test set

cnn_model.evaluate(X_test,y_test)

In [ ]:
# display 12 images with prediction and actual labels in test set

fig, axs = plt.subplots(3, 4, figsize=(15, 15))

i = 0
for x in range(3):
    for y in range(4):
        prediction = cnn_model.predict(X_test[i][None, ...], verbose=0)[0]
        
        axs[x][y].set_xticks([])
        axs[x][y].set_yticks([])
        axs[x][y].set_xlabel(f'prediction: {class_names[np.argmax(prediction)]} | label: {class_names[np.argmax(y_test[i])]}')
        
        axs[x][y].imshow(X_test[i])
        
        i += 1
plt.show()

In [ ]:
# save the trained model

# cnn_model.save('emotion_detector.keras')